In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
def loadDataset(name=Dataset, path='/content/'):
  transform = transforms.Compose([transforms.ToTensor()])
  if name == 'MNIST': return loadMNIST(path, transform)
  if name == 'Fashion': return loadFashion(path, transform)
  if name == 'SVHN': return loadSVHN(path, transform)
  if name == 'CIFAR': return loadCIFAR(path, transform)
  raise 'Dataset not found'

In [ ]:
def loadMNIST(path, transform):
  return datasets.MNIST(path, True, transform, download=True), \
         datasets.MNIST(path, False, transform, download=True)

def loadFashion(path, transform):
  return datasets.FashionMNIST(path, True, transform, download=True), \
         datasets.FashionMNIST(path, False, transform, download=True)

def loadSVHN(path, transform):
  return datasets.SVHN(path, 'train', transform, download=True), \
         datasets.SVHN(path, 'test', transform, download=True)

def loadCIFAR(path, transform):
  return datasets.CIFAR10(path, True, transform, download=True), \
         datasets.CIFAR10(path, False, transform, download=True)

In [ ]:
import torch
class DatasetGPU(torch.utils.data.Dataset):
  def __init__(self, dataset):
    self.length = dataset.__len__()
    loader = DataLoader(dataset, self.length)
    for images, labels in loader:
      self.images = images.cuda()
      self.labels = labels.cuda()
  def __len__(self):
    return self.length
  def __getitem__(self, idx):
    return self.images[idx], self.labels[idx]

In [ ]:
class DataLoaderGPU(DataLoader):
  def __init__(self, dataset, batch_size, shuffle=False):
    super().__init__(DatasetGPU(dataset), batch_size, shuffle)

In [ ]:
def loadTester(testset, batch=batch):
  global in_channels, num_classes
  if len(testset) == 3: testset, in_channels, num_classes = testset
  return in_channels, num_classes, DataLoaderGPU(testset, batch * 2)

In [ ]:
def saveAccuracies(path, clients, postfix=''):
  accuracies = [c.accuracy for c in clients]
  path = f'{path}/data/{Dataset}-{Model}-{Prefix}-{num_clients}-{postfix}.acc'
  torch.save(accuracies, path)
  print(path)